# 1. Import and Install Dependencies

In [1]:
!pip install tensorflow opencv-python mediapipe scikit-learn matplotlib

Defaulting to user installation because normal site-packages is not writeable


In [31]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# 2. Keypoints using MP Holistic

In [3]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [5]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections

In [6]:
def draw_styled_landmarks(image, results):

    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 


In [7]:
cap = cv2.VideoCapture(0)
HEIGHT = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) # webcam video frame height
WIDTH = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) # webcam video frame width
FPS = int(cap.get(cv2.CAP_PROP_FPS)) # webcam video fram rate 
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [8]:
draw_styled_landmarks(frame, results)

In [36]:
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

# 3. Extract Keypoint Values

In [28]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    return pose

# 4. Setup Folders for Collection

In [8]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join(os. getcwd(),'data') 
print(DATA_PATH)

# make directory if it does not exist yet
if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH)

# Actions/exercises that we try to detect
actions = np.array(['pushup', 'squat', 'lunge', 'noactions'])
num_classes = len(actions)

# How many videos worth of data
no_sequences = 75

# Videos are going to be this many frames in length
sequence_length = 30

# Folder start
# Change this to collect more data and not lose previously collected data
start_folder = 101


C:\Users\Fatema\Documents\benAI\gym_pose_trainer\data


In [45]:
for action in actions: 
    dirmax = np.max(np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int))
    for sequence in range(1,no_sequences+1):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(dirmax+sequence)))
        except:
            pass

# 5. Collect Keypoint Values for Training and Testing

In [7]:
# Colors associated with each exercise (e.g., pushhups are denoted by blue, squats are denoted by orange, etc.)
colors = [(245,117,16), (117,245,16), (16,117,245),(76,0,230)]
#add one more color

In [47]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(start_folder, start_folder+no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [48]:
cap.release()
cv2.destroyAllWindows()

# 6. Preprocess Data and Create Labels and Features

In [8]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [9]:
label_map = {label:num for num, label in enumerate(actions)}

NameError: name 'actions' is not defined

In [51]:
label_map

In [52]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [54]:
# Make sure first dimensions of arrays match
X = np.array(sequences)
y = to_categorical(labels).astype(int)
print(X.shape, y.shape)

In [55]:
# np.array(sequences).shape

In [56]:
# np.array(labels).shape

# 7. Build and Train LSTM Neural Network

In [10]:
#some of these imports might not be needed, added in the testing phase

from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score, classification_report
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import KFold

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

from tensorflow.keras.models import Sequential, Model

from tensorflow.keras.layers import (LSTM, Dense, Concatenate, Attention, Dropout, Softmax,
                                     Input, Flatten, Activation, Bidirectional, Permute, multiply, 
                                     ConvLSTM2D, MaxPooling3D, TimeDistributed, Conv2D, MaxPooling2D)

from scipy import stats

# disable some of the tf/keras training warnings 
os.environ['TF_CPP_MIN_LOG_LEVEL'] = "3"
tf.get_logger().setLevel("ERROR")
tf.autograph.set_verbosity(1)

# suppress untraced functions warning
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)

In [13]:
actions = np.array(['pushup', 'squat', 'lunge', 'noactions'])

Few techniques were tried, two models that worked best where this sequential model and the sasme with a k-fold validation

In [70]:
# model = Sequential()
# model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
# model.add(LSTM(128, return_sequences=True, activation='relu'))
# model.add(LSTM(64, return_sequences=False, activation='relu'))
# model.add(Dense(64, activation='relu'))
# model.add(Dense(32, activation='relu'))
# model.add(Dense(actions.shape[0], activation='softmax'))

In [72]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [73]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [11]:
kfold = KFold(n_splits=5, shuffle=True, random_state=42)


In [14]:
lstm = Sequential()
lstm.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,132)))
lstm.add(LSTM(128, return_sequences=True, activation='relu'))
lstm.add(LSTM(64, return_sequences=False, activation='relu'))
lstm.add(Dense(64, activation='relu'))
lstm.add(Dense(32, activation='relu'))
lstm.add(Dense(actions.shape[0], activation='softmax'))
print(lstm.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 30, 64)            50432     
                                                                 
 lstm_4 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dense_3 (Dense)             (None, 32)                2080      
                                                                 
 dense_4 (Dense)             (None, 4)                 132       
                                                                 
Total params: 205028 (800.89 KB)
Trainable params: 205

In [15]:
lstm.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [50]:
model.fit(X_train, y_train, epochs=100, callbacks=[tb_callback])

# 8. Make Predictions

In [52]:
res = lstm.predict(X_test)

In [53]:
actions[np.argmax(res[1])]

In [54]:
actions[np.argmax(y_test[1])]

# 9. Save Weights

In [55]:
lstm.save('model.keras')

In [56]:
del model

In [16]:
lstm.load_weights('model.keras')

# 10. Evaluation using Confusion Matrix and Accuracy

In [17]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [59]:
yhat = model.predict(X_test)

In [60]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [61]:
multilabel_confusion_matrix(ytrue, yhat)

In [62]:
accuracy_score(ytrue, yhat)

In [ ]:
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt

# Get predictions for the entire dataset
predictions = lstm.predict(X)
true_labels = np.argmax(y, axis=1)  # Convert one-hot encoded true labels to integer labels if necessary
predicted_labels = np.argmax(predictions, axis=1)  # Convert one-hot encoded predicted labels to integer labels if necessary

# Compute the confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

from sklearn.metrics import confusion_matrix
import seaborn as sns

# Get predictions for the entire dataset
predictions = lstm.predict(X)
true_labels = np.argmax(y, axis=1)  # Convert one-hot encoded true labels to integer labels if necessary
predicted_labels = np.argmax(predictions, axis=1)  # Convert one-hot encoded predicted labels to integer labels if necessary

# Compute the confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.title("Confusion Matrix")
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

# Calculate evaluation metrics
precision = precision_score(true_labels, predicted_labels, average='weighted')
recall = recall_score(true_labels, predicted_labels, average='weighted')
accuracy = accuracy_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels, average='weighted')

# Print the evaluation metrics
print("\nPrecision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("Accuracy: {:.4f}".format(accuracy))
print("F1 Score: {:.4f}".format(f1))

# 11. Test in Real Time

In [120]:
from scipy import stats

In [18]:
colors = [(245,117,16), (117,245,16), (16,117,245),(76,0,230)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [19]:
actions = np.array(['pushup', 'squat', 'lunge', 'noactions'])
num_classes = len(actions)

In [20]:
colors = [(245,117,16), (117,245,16), (16,117,245),(76,0,230)]
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = lstm.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

# 12. Find issues in form

In [21]:
def calculate_angle(a,b,c):
    """
    Computes 3D joint angle inferred by 3 keypoints and their relative positions to one another, under 180
    
    """
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [22]:
def calculate_angle_pushup(a,b,c):
    """
    Computes 3D joint angle inferred by 3 keypoints and their relative positions to one another, not bound by 180
    
    """
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
        
    return angle 

In [23]:
def get_coordinates(landmarks, mp_pose, side, joint):
    """
    Retrieves x and y coordinates of a particular keypoint from the pose estimation model
         
     Args:
         landmarks: processed keypoints from the pose estimation model
         mp_pose: Mediapipe pose estimation model
         side: 'left' or 'right'. Denotes the side of the body of the landmark of interest.
         joint: 'shoulder', 'elbow', 'wrist', 'hip', 'knee', or 'ankle'. Denotes which body joint is associated with the landmark of interest.
    
    """
    coord = getattr(mp_pose.PoseLandmark,side.upper()+"_"+joint.upper())
    x_coord_val = landmarks[coord.value].x
    y_coord_val = landmarks[coord.value].y
    return [x_coord_val, y_coord_val]        

In [24]:
def viz_joint_angle(image, angle, joint):
    """
    Displays the joint angle value near the joint within the image frame
    
    """
    cv2.putText(image, str(int(angle)), 
                   tuple(np.multiply(joint, [640, 480]).astype(int)), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                        )
    return

In [25]:
def prob_viz(res, actions, input_frame, colors, threshold=0.5):
    """
    This function displays the most probable exercise class above the threshold
    on the input frame, with its probability as a percentage, in the top right corner.
    """
    output_frame = input_frame.copy()
    frame_height, frame_width = input_frame.shape[:2]  # Get the height and width of the frame

    # Find the index of the maximum probability that is also above the threshold
    max_prob_index = None
    max_prob = threshold
    for num, prob in enumerate(res):
        if prob > max_prob:
            max_prob = prob
            max_prob_index = num

    # If an action with a probability above the threshold was found, display it
    if max_prob_index is not None:
        action_text = f'{actions[max_prob_index]}: {max_prob*100:.2f}%'
        text_size = cv2.getTextSize(action_text, cv2.FONT_HERSHEY_SIMPLEX, 0.75, 2)[0]
        text_width = text_size[0]

        # Position the text in the top right corner, with a small margin
        fixed_position = (frame_width - text_width - 10, 30)  # Adjust the horizontal position as needed

        cv2.putText(output_frame, action_text, fixed_position, cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 0), 2, cv2.LINE_AA)

    return output_frame

In [26]:
def count_reps_and_feedback(image, current_action, landmarks, mp_pose):
    """
    Counts repetitions of each exercise, find form errors and display. Global count and stage (i.e., state) variables are updated within this function.
    
    """

    global  squat_counter, squat_stage, pushup_counter, pushup_stage, lunge_counter, lunge_stage
    
    start_x, start_y = 10, 30
    line_height = 30  # Increase line height for better spacing
    text_font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.65
    font_thickness = 2
    text_color = (255, 255, 255)  # White
    rectangle_bgr_color = (0, 0, 255)  # Red
    action_text = f'{current_action}: '

    
    if current_action == 'squat':
        # Get coords
        # left side
        # Detect mistakes
        feedback = []
        left_shoulder = get_coordinates(landmarks, mp_pose, 'left', 'shoulder')
        left_hip = get_coordinates(landmarks, mp_pose, 'left', 'hip')
        left_knee = get_coordinates(landmarks, mp_pose, 'left', 'knee')
        left_ankle = get_coordinates(landmarks, mp_pose, 'left', 'ankle')
        left_toe = get_coordinates(landmarks, mp_pose, 'left', 'foot_index')
        # right side
        right_shoulder = get_coordinates(landmarks, mp_pose, 'right', 'shoulder')
        right_hip = get_coordinates(landmarks, mp_pose, 'right', 'hip')
        right_knee = get_coordinates(landmarks, mp_pose, 'right', 'knee')
        right_ankle = get_coordinates(landmarks, mp_pose, 'right', 'ankle')
        right_toe = get_coordinates(landmarks, mp_pose, 'right', 'foot_index')
        
        # Calculate knee angles
        left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
        right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)
        
        # Calculate hip angles
        left_hip_angle = calculate_angle(left_shoulder, left_hip, left_knee)
        right_hip_angle = calculate_angle(right_shoulder, right_hip, right_knee)

        


        
        # Squat counter logic
        thr = 160
        if (left_knee_angle < thr) and (right_knee_angle < thr) and (left_hip_angle < thr) and (right_hip_angle < thr):
            squat_stage = "down"
        if (left_knee_angle > thr) and (right_knee_angle > thr) and (left_hip_angle > thr) and (right_hip_angle > thr) and (squat_stage =='down'):
            squat_stage='up'
            squat_counter += 1


        if (left_knee_angle > 120 and left_knee_angle>160 ) or (right_knee_angle > 120 and right_knee_angle>160 ) :
            feedback.append("Go lower")
        if (left_knee_angle < 65) or (right_knee_angle < 65):
            feedback.append("too low")

                
        viz_joint_angle(image, left_knee_angle, left_knee)
        viz_joint_angle(image, left_hip_angle, left_hip)
        
                # Combine feedback into a single string
        combined_feedback = ', '.join(feedback)

        # Display feedback on the image

# Iterate through each feedback message, draw a rectangle and then the text
    
    elif current_action == 'pushup':
        # Get coordinates for the shoulders, elbows, wrists, hips, and knees
        left_shoulder = get_coordinates(landmarks, mp_pose, 'left', 'shoulder')
        left_elbow = get_coordinates(landmarks, mp_pose, 'left', 'elbow')
        left_wrist = get_coordinates(landmarks, mp_pose, 'left', 'wrist')
        left_hip = get_coordinates(landmarks, mp_pose, 'left', 'hip')
        left_knee = get_coordinates(landmarks, mp_pose, 'left', 'knee')

        right_shoulder = get_coordinates(landmarks, mp_pose, 'right', 'shoulder')
        right_elbow = get_coordinates(landmarks, mp_pose, 'right', 'elbow')
        right_wrist = get_coordinates(landmarks, mp_pose, 'right', 'wrist')
        right_hip = get_coordinates(landmarks, mp_pose, 'right', 'hip')
        right_knee = get_coordinates(landmarks, mp_pose, 'right', 'knee')

        # Calculate elbow angles
        left_elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
        right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)

        # Body alignment angles for sagging or piking hips
        left_body_angle = calculate_angle_pushup(left_shoulder, left_hip, left_knee)
        right_body_angle = calculate_angle_pushup(right_shoulder, right_hip, right_knee)
        


        left_hip_elbow_angle = calculate_angle_pushup(left_hip, left_shoulder, left_elbow)
        right_hip_elbow_angle = calculate_angle_pushup(right_hip, right_shoulder, right_elbow)

       # Pushup counter logic
        thr = 160  # Threshold for considering an elbow fully extended
        if (left_elbow_angle > thr) and (right_elbow_angle > thr) and pushup_stage != "down":
            pushup_stage = "down"
        elif (left_elbow_angle < thr) and (right_elbow_angle < thr) and pushup_stage == "down":
            pushup_stage = "up"
            pushup_counter += 1
            
        # Detect mistakes@
        feedback = []


#         Mistake 1: Sagging Hips
        if left_body_angle > 190 or right_body_angle > 190:   # Adjust thresholds as needed
            feedback.append("Tighten core, no sagging hips")

        # Mistake 2: Piking Hips
        if left_body_angle < 160 or right_body_angle < 160: # Adjust thresholds as needed
            feedback.append("Lower hips to straight line")


        # Combine feedback into a single string
        combined_feedback = ', '.join(feedback)



# Iterate through each feedback message, draw a rectangle and then the text
        for i, single_feedback in enumerate(feedback):
            (text_width, text_height), _ = cv2.getTextSize(single_feedback, text_font, font_scale, font_thickness)
            # Calculate the box coordinates to place the background rectangle
            box_coords = ((start_x, start_y + i * line_height - text_height - 4), 
                          (start_x + text_width + 4, start_y + i * line_height + 6))
            cv2.rectangle(image, box_coords[0], box_coords[1], rectangle_bgr_color, cv2.FILLED)
            cv2.putText(image, single_feedback, (start_x, start_y + i * line_height), 
                          text_font, font_scale, text_color, font_thickness, cv2.LINE_AA)
        

                
    elif current_action == 'lunge':
        feedback = []
        # Assuming you are tracking the front leg for lunges
        # Get coords for the front leg (choose left or right based on your setup)
        front_shoulder = get_coordinates(landmarks, mp_pose, 'left', 'shoulder')
        front_hip = get_coordinates(landmarks, mp_pose, 'left', 'hip')
        front_knee = get_coordinates(landmarks, mp_pose, 'left', 'knee')
        front_ankle = get_coordinates(landmarks, mp_pose, 'left', 'ankle')

        # Get coords for the back leg (choose left or right based on your setup)
        back_hip = get_coordinates(landmarks, mp_pose, 'right', 'hip')
        back_knee = get_coordinates(landmarks, mp_pose, 'right', 'knee')
        back_ankle = get_coordinates(landmarks, mp_pose, 'right', 'ankle')

        # Calculate knee angles
        front_knee_angle = calculate_angle(front_hip, front_knee, front_ankle)
        back_knee_angle = calculate_angle(back_hip, back_knee, back_ankle)

        # Lunge counter logic
        # Define your threshold angle for detecting a lunge
        thr = 140  # This is an example threshold; you may need to adjust this based on your observations
        if front_knee_angle < thr and back_knee_angle > thr:  # This is a simplification; adjust as needed
            lunge_stage = "down"
        if front_knee_angle > thr and lunge_stage == "down":
            lunge_stage = "up"
            lunge_counter += 1

        # Viz joint angles for the front leg
        viz_joint_angle(image, front_knee_angle, front_knee)

    else:
        pass
    
    for i, single_feedback in enumerate(feedback):
        (text_width, text_height), _ = cv2.getTextSize(single_feedback, text_font, font_scale, font_thickness)
                # Calculate the box coordinates to place the background rectangle
        box_coords = ((start_x, start_y + i * line_height - text_height - 4), 
                              (start_x + text_width + 4, start_y + i * line_height + 6))
        cv2.rectangle(image, box_coords[0], box_coords[1], rectangle_bgr_color, cv2.FILLED)
        cv2.putText(image, single_feedback, (start_x, start_y + i * line_height), 
                              text_font, font_scale, text_color, font_thickness, cv2.LINE_AA)

In [37]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
sequence = []
predictions = []
res = []
threshold = 0.5 # minimum confidence to classify as an action/exercise
current_action = ''
model_name = 'LSTM'

squat_counter = 0
squat_stage = None
pushup_counter = 0
pushup_stage = None
lunge_counter = 0
lunge_stage = None

cap = cv2.VideoCapture(0)

#saving video
fourcc = cv2.VideoWriter_fourcc('M','J','P','G') # video compression format
HEIGHT = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) # webcam video frame height
WIDTH = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) # webcam video frame width
FPS = int(cap.get(cv2.CAP_PROP_FPS)) # webcam video fram rate 

video_name = os.path.join(os.getcwd(),f"{model_name}opneLab3.avi")
out = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*"MJPG"), FPS, (WIDTH,HEIGHT))

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        ret, frame = cap.read()

        image, results = mediapipe_detection(frame, holistic)
        
        draw_landmarks(image, results)
        
        sequence_length = 30

        keypoints = extract_keypoints(results)        
        sequence.append(keypoints)      
        sequence = sequence[-sequence_length:]


        if len(sequence) == sequence_length:
            res = lstm.predict(np.expand_dims(sequence, axis=0), verbose=0)[0]           
            predictions.append(np.argmax(res))
            current_action = actions[np.argmax(res)]
            confidence = np.max(res)
            
            if confidence < threshold:
                current_action = ''

            image = prob_viz(res, actions, image, colors)
            
            try:
                landmarks = results.pose_landmarks.landmark
                count_reps_and_feedback(
                    image, current_action, landmarks, mp_holistic)
            except:
                pass

            top_left_x = 20  # Move right by 50 pixels
            top_left_y = 440  # Move down by 100 pixels
            bottom_right_x = top_left_x + 640  # Keep the same width
            bottom_right_y = top_left_y + 40  # Keep the same height

            cv2.rectangle(image, (top_left_x, top_left_y), (bottom_right_x, bottom_right_y), (117,245,16), -1)

            cv2.putText(image, 'Squats ' + str(squat_counter) + "", (490,470), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, 'Pushups ' + str(pushup_counter) + "  ", (310,470), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, 'Lunges ' + str(lunge_counter) + " ", (150,470), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
         
         
        cv2.imshow('OpenCV Feed', image)

        if ret == True:
            out.write(image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    out.release()
    cv2.destroyAllWindows()

In [36]:
    cap.release()
    out.release()
    cv2.destroyAllWindows()